In [22]:
import re
import pandas as pd

In [23]:
raw_7_df = pd.read_excel(
    "./../raw_data/제_7_전국동시지방선거_개표/20180619-7지선-02-(구시군의장)_읍면동별개표자료.xlsx",
    header=[0],
)

In [24]:
df = raw_7_df.copy()

In [25]:
df.head()

,선거종류,시도,선거구명,시도명,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,무효투표수,기권수
0,구·시·군의 장선거,서울특별시,종로구,NaN,NaN,NaN,NaN,NaN,NaN,더불어민주당_x000D_\n김영종,자유한국당_x000D_\n이숙연,바른미래당_x000D_\n김복동,_x000D_\n,_x000D_\n,_x000D_\n,_x000D_\n,계,NaN,NaN
1,NaN,서울특별시,종로구,합계,NaN,NaN,NaN,134963.0,81195.0,51305,19628,8765,0,0,0,0,79698,1497.0,53768.0
2,NaN,서울특별시,종로구,서울특별시,종로구,계,NaN,134963.0,81195.0,51305,19628,8765,0,0,0,0,79698,1497.0,53768.0
3,NaN,서울특별시,종로구,서울특별시,종로구,거소투표,NaN,145.0,138.0,84,28,19,0,0,0,0,131,7.0,7.0
4,NaN,서울특별시,종로구,서울특별시,종로구,관외사전투표,NaN,8516.0,8493.0,6053,1509,749,0,0,0,0,8311,182.0,23.0


In [26]:
# get splited_index
_df = df.dropna(subset=["선거종류"])
index_zip_list = list(zip(_df.index, _df.index[1:]))

In [27]:
# replacer
_replace_nan = lambda x: "" if type(x) != str else x
_replace_unnamed = lambda x: "후보자별 득표수" if x.startswith("Unnamed") else x
_replace_escape = lambda x: x.replace("_x000D_\n", "_")
_replace = lambda x: _replace_escape(_replace_unnamed(_replace_nan(x)))

_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace, x))

In [28]:
def replace_position(df, lib, con, etc):
    """
    lib : 진보당
    con : 보수당
    etc : 나머지당
    """
    df["lib"] = df.loc[:, lib].sum(axis=1)
    df["con"] = df.loc[:, con].sum(axis=1)
    df["etc"] = df.loc[:, etc].sum(axis=1)
    df.drop(columns=(lib + con + etc), inplace=True)
    return df


In [29]:
arg_list = list()
partial_df_list = list()
for index_zip in index_zip_list:
    # partial data
    partial_df = df.iloc[index_zip[0] : index_zip[1], :]

    # preproc columns
    multi_cols = [(col_a, col_b) for col_a, col_b in zip(partial_df.columns, partial_df.iloc[0, :])]

    flatten_cols = list(map(func, multi_cols))
    partial_df.columns = flatten_cols

    # drop duplicated data
    partial_df = partial_df.iloc[1:, 3:]

    # filter agg / etcs
    partial_df = partial_df[
        (partial_df["시도명"] != "합계")
        & (partial_df["구시군명"] != "합계")
        & ~(
            partial_df["읍면동명"].isin(
                ["합계", "거소투표", "관외사전투표", "재외투표", "잘못 투입·구분된 투표지"]
            )
        )
        & (partial_df["구분"] == "소계")
    ]
    partial_df = partial_df.loc[:, ~partial_df.columns.duplicated()]

    # renaming
    rename_dict = {
        "시도명": "loc_1",
        "구시군명": "loc_2",
        "읍면동명": "loc_3",
    }

    partial_df.rename(columns=rename_dict, inplace=True)
    partial_df.drop(columns=["구분", "후보자별 득표수"], inplace=True, errors="ignore")

    votes = list(filter(lambda x: x.startswith("후보자별 득표수"), partial_df.columns))
    partial_df.loc[:, votes] = partial_df.loc[:, votes].astype(int)
    lib = list(filter(lambda x: x.startswith("후보자별 득표수_더불어민주당"), partial_df.columns))
    con = list(filter(lambda x: x.startswith("후보자별 득표수_자유한국당"), partial_df.columns))
    etc = list(set(votes) - (set(lib) | set(con)))
    partial_df = replace_position(partial_df, lib, con, etc)

    # save
    argument = partial_df["loc_1"].iloc[0] + "_" + partial_df["loc_2"].iloc[0]
    arg_list.append(argument)
    partial_df_list.append(partial_df)

In [30]:
구청군청장_7 = pd.concat(partial_df_list, axis=0)
구청군청장_7.head()

,loc_1,loc_2,loc_3,선거인수,투표수,무효투표수,기권수,lib,con,etc
5,서울특별시,종로구,청운효자동,9719.0,6046.0,119.0,3673.0,3862,1500,6492
8,서울특별시,종로구,사직동,8106.0,4966.0,94.0,3140.0,2853,1536,5355
11,서울특별시,종로구,삼청동,2307.0,1283.0,17.0,1024.0,744,395,1393
14,서울특별시,종로구,부암동,7997.0,4777.0,89.0,3220.0,2953,1320,5103
17,서울특별시,종로구,평창동,15012.0,8814.0,111.0,6198.0,4929,3005,9472


In [31]:
구청군청장_7.to_csv("./1st_preproc_data/구청군청장_7.csv")

In [32]:
raw_8_df = pd.read_excel(
    "./../raw_data/제_8_전국동시지방선거_개표/구시군장선거.xlsx", header=[0, 1], thousands=","
)

In [33]:
df = raw_8_df.copy()

In [34]:
# preproc columns
_replace_unnamed = lambda x: "" if x.startswith("Unnamed") else x
_replace_candidate = lambda x: "" if re.match("^후보\d+", x) else x
_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace_unnamed, map(_replace_candidate, x)))

flatten_cols = list(map(func, df.columns))
df.columns = flatten_cols

In [35]:
# get splited_index
index_zip_list = list()
for arg in df["선거구(구시군)"].unique():
    _index = df[df["선거구(구시군)"] == arg].index
    index_zip_list.append((min(_index), max(_index)))

In [36]:
_replace_nan = lambda x: "" if type(x) != str else x
_replace_unnamed = lambda x: "후보자별 득표수" if x.startswith("Unnamed") else x
_replace_escape = lambda x: x.replace("\n", "_")
_replace = lambda x: _replace_escape(_replace_unnamed(_replace_nan(x)))

_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace, x))

In [37]:
arg_list = list()
partial_df_list = list()

for index_zip in index_zip_list:
    partial_df = df.iloc[index_zip[0] : index_zip[1], :]

    # preproc columns
    multi_cols = [(col_a, col_b) for col_a, col_b in zip(partial_df.columns, partial_df.iloc[0, :])]
    flatten_cols = list(map(func, multi_cols))
    partial_df.columns = flatten_cols

    #
    partial_df = partial_df.iloc[1:, :]

    #
    partial_df = partial_df[
        ~(
            partial_df["읍면동명"].isin(
                ["합계", "거소투표", "관외사전투표", "재외투표", "잘못 투입·구분된 투표지"]
            )
        )
        & (partial_df["구분"] == "소계")
    ]

    #
    _replace_default = lambda x: x.replace("_선거인수", "")
    partial_df.columns = [_replace_default(col) for col in partial_df.columns]
    partial_df.columns = ["loc_1", "loc_drop", "loc_2", "loc_3"] + list(partial_df.columns)[4:]
    partial_df = partial_df.drop(columns=["loc_drop", "구분", "후보자별 득표수"], errors="ignore")

    votes = list(filter(lambda x: x.startswith("후보자별 득표수_"), partial_df.columns))
    partial_df.loc[:, votes] = partial_df.loc[:, votes].astype(int)
    lib = list(filter(lambda x: x.startswith("후보자별 득표수_더불어민주당"), partial_df.columns))
    con = list(filter(lambda x: x.startswith("후보자별 득표수_국민의힘"), partial_df.columns))
    etc = list(set(votes) - (set(lib) | set(con)))
    partial_df = replace_position(partial_df, lib, con, etc)

    #
    # save
    argument = partial_df["loc_1"].iloc[0] + "_" + partial_df["loc_2"].iloc[0]
    arg_list.append(argument)
    partial_df_list.append(partial_df)

In [38]:
구청군청장_8 = pd.concat(partial_df_list, axis=0)
구청군청장_8.head()

,loc_1,loc_2,loc_3,선거인수,투표수,계,무효투표수,기권수,lib,con,etc
4,서울특별시,종로구,청운효자동,9179.0,4887.0,4844.0,43.0,4292.0,2404,2361,79
7,서울특별시,종로구,사직동,8057.0,4450.0,4394.0,56.0,3607.0,1755,2563,76
10,서울특별시,종로구,삼청동,1962.0,1000.0,991.0,9.0,962.0,419,547,25
13,서울특별시,종로구,부암동,7693.0,4137.0,4102.0,35.0,3556.0,1878,2169,55
16,서울특별시,종로구,평창동,14730.0,7930.0,7866.0,64.0,6800.0,2913,4874,79


In [39]:
구청군청장_8.to_csv("./1st_preproc_data/구청군청장_8.csv")